In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.8


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
df['predicted_duration'] = y_pred

In [11]:
df.predicted_duration.describe()

count    3.316216e+06
mean     1.420387e+01
std      6.247490e+00
min     -1.160751e+01
25%      1.096814e+01
50%      1.197354e+01
75%      1.413772e+01
max      7.498748e+01
Name: predicted_duration, dtype: float64

In [12]:
year = 2023
month = 3

In [13]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,predicted_duration,ride_id
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,...,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000,16.245906,2023/03_0
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,...,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333,26.134796,2023/03_1
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,...,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667,11.884264,2023/03_2
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,...,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667,11.997720,2023/03_3
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,...,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333,10.234486,2023/03_4


In [15]:
df_result = df[['ride_id', 'predicted_duration']]

In [16]:
df_result.head()

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [17]:
output_file = 'df_result.parquet'

In [18]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [21]:
!ls -l --block-size=M

total 66M
-rw-r--r-- 1 Admin 197121 66M Jun 12 00:30 df_result.parquet
-rw-r--r-- 1 Admin 197121  1M Jun 12 00:10 model.bin
-rw-r--r-- 1 Admin 197121  1M Jun 12 00:26 note.md
-rw-r--r-- 1 Admin 197121  1M Jun 12 00:24 Pipfile
-rw-r--r-- 1 Admin 197121  1M Jun 12 00:24 Pipfile.lock
-rw-r--r-- 1 Admin 197121  1M Jun 12 00:32 starter.ipynb
